# Decimal type in python3

## 0. What is the problem with built-in float numbers?

In [1]:
x = 0.1 + 0.1 + 0.1

print(f"0.1 + 0.1 + 0.1 = {x}")
print(f"Is x = 0.3? {x == 0.3}")

0.1 + 0.1 + 0.1 = 0.30000000000000004
Is x = 0.3? False


## 1. Solution

In [2]:
from decimal import Decimal, getcontext

x = Decimal('0.1') + Decimal('0.1') + Decimal('0.1')

print(f"Decimal('0.1') + Decimal('0.1') + Decimal('0.1') = {x}")
print(f"Is x = 0.3? {float(x) == 0.3}")

Decimal('0.1') + Decimal('0.1') + Decimal('0.1') = 0.3
Is x = 0.3? True


## 2. Performance

### 2.0 Simple function with built-in float

In [3]:
def math_multiplication_power_loop(x,y):
    number = []
    
    for i in range(40):
        number.append((x*y)**i)
    
    return sum(number)

import time

start = time.time()
for i in range(100_000):
    result = math_multiplication_power_loop(12344, 5567.8)

print(f"Elapsed time: {time.time() - start}s")
print(f"Result: {result}")

Elapsed time: 1.8481860160827637s
Result: 4.450840747566237e+305


### 2.1 Simple function with Decimal

In [4]:
def math_multiplication_power_loop_decimal(x,y):
    number = []
    
    for i in range(40):
        number.append((Decimal(x)*Decimal(y))**Decimal(i))
    
    return sum(number)

start = time.time()
for i in range(100_000):
    result = math_multiplication_power_loop_decimal(12344, 5567.8)
    
print(f"Elapsed time: {time.time() - start}s")
print(f"Result: {result}")

Elapsed time: 12.986524105072021s
Result: 4.450840747566235891918380373E+305


#### 2.1.0 Simple function with Decimal with higher precision

In [5]:
from decimal import getcontext

getcontext().prec = 50

start = time.time()
for i in range(100_000):
    result = math_multiplication_power_loop_decimal(12344, 5567.8)
    
print(f"Elapsed time: {time.time() - start}s")
print(f"Result: {result}")

Elapsed time: 16.095144510269165s
Result: 4.4508407475662358919183803654064648855495046679648E+305


### 2.2 Simple function with njit

In [4]:
from numba import njit
import time

@njit(fastmath=True)
def math_multiplication_power_loop_njit(x,y):
    number = []
    
    for i in range(40):
        number.append((x*y)**i)
    
    return sum(number)

start = time.time()
for i in range(100_000):
    result = math_multiplication_power_loop_njit(12344, 5567.8)
   
print(f"Elapsed time: {time.time() - start}s")
print(f"Result: {result}")

Elapsed time: 0.5792031288146973s
Result: 4.450840747566245e+305


### 2.3 Simple function with njit and numpy

In [7]:
import numpy as np

@njit(fastmath=True)
def math_multiplication_power_loop_njit_numpy(x,y):
    number = np.empty(shape=(1,))
    
    for i in range(40):
        number = np.append(number, (x*y)**i )
    
    return np.sum(number)

start = time.time()
for i in range(1_000_000):
    result = math_multiplication_power_loop_njit_numpy(12344, 5567.8)

print(f"Elapsed time: {time.time() - start}s")
print(f"Result: {result}")

Elapsed time: 26.85703754425049s
Result: 4.450840747566245e+305


## 3. Results

1. Your mathematical function is fast
   - If basic precision is good for you, do not change it!
   - If basic precision is not enough, use Decimal. Prepare for small performance lose.
2. Your mathematical function is slow
    - If basic precision is good for you, use njit. Faster, same result.
    - If basic precision is not enough, use Decimal. You cannot make it faster with njit.
3. Numpy do not have higher precision then built-in float